In [4]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
import csv
from datetime import datetime,timedelta
import cv2


In [5]:
plate = 13
date_init = datetime(2020,7,1,19,57)
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(8)]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [ ]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [24]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(f'Data/graph_{date}_{plate}_full_labeled.csv',
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [ ]:
from_tip_growth_pattern=[]
for date in dates[:-1]:
    from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(get_path(date,plate,True,extension='_connection.csv'))))

In [43]:
from_tip_growth_pattern=[]
for date in dates[:-1]:
    from_tip_growth_pattern.append(from_connection_tab(pd.read_csv(f'Data/graph_{date}_{plate}_connection.csv')))

In [25]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [26]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [27]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20800, 46000)))

In [28]:
factor = 5
final_pictures = [compress_skeleton(skeletons[i],factor) for i in range(len(skeletons))]

In [44]:
connections = [c[0] for c in from_tip_growth_pattern]
growth_patterns = [c[1] for c in from_tip_growth_pattern]

In [45]:
growths = [{tip : sum([len(branch) for branch in growth_pattern[tip]]) for tip in growth_pattern.keys()} for growth_pattern in growth_patterns]

In [46]:
growing_tips=[[node for node in growths[i].keys() if growths[i][node]>=80] for i in range(len(growths))]

In [55]:
degree1_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)==1 and node not in nx_graph_clean[i+1].nodes] for i,nx_graph in enumerate(nx_graph_clean[:-1])]
t=3
tp1=t+1
plot_t_tp1(degree1_nodes[t],degree1_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
t=3
tp1=t+1
plot_t_tp1(origins[t],anastomosiss[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],
           compress=5,time='4h',relabel_tp1=lambda node : find_origin_tip(node,connections[t]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
t=4
tp1=t+1
plot_t_tp1(growing_tips[t],growing_tips[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

In [56]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht,from_tip,pos_tm1):
    anastomosis=[]
    origins=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    number_anastomosis = 0
    def dist_branch(node,nx_graph,pos):
        mini=np.inf
        for edge in nx_graph.edges:
            pixel_list=nx_graph.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos[node]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos[node]),axis=1))
                if distance<mini:
                    mini=distance
        return(mini)
    def count_neighbors_is_from_root(equ_list,nx_graph,root):
        count=0
        for neighbor in nx_graph.neighbors(root):
            if neighbor in equ_list:
                count+=1
        return(count)
    for tip in tips:
#         print(tip)
        consequence = from_tip[tip]
        for node in consequence:
            if node in nx_grapht.nodes and nx_grapht.degree(node)>=3 and count_neighbors_is_from_root(consequence,nx_grapht,node)<2:
#                 if node==2753:
#                     print(count_neighbors_is_from_root(consequence,nx_grapht,node))
#                     print(list(nx_grapht.neighbors(node)))
                anastomosis.append(node)
                origins.append(tip)
                number_anastomosis+=1
        if tip not in nx_grapht.nodes and dist_branch(tip,nx_grapht,pos_tm1)<=30:
            origins.append(tip)
            number_anastomosis+=1/2
    return(anastomosis,origins,number_anastomosis)

In [51]:
def find_origin_tip(node,from_tip):
    for tip in from_tip.keys():
        if node in from_tip[tip]:
            return(tip)

In [57]:
anastomosiss=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i],poss[i])[0] for i in range (len(dates)-1)]
origins=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i],poss[i])[1] for i in range (len(dates)-1)]
number_an = [pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i],poss[i])[2] for i in range (len(dates)-1)]

In [62]:
plt.close('all')

In [60]:
node_interest=2152
pos_problem=poss[t][node_interest]
xbegin=pos_problem[0]-500
ybegin=pos_problem[1]-500
xend=pos_problem[0]+500
yend=pos_problem[1]+500
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])
kernel = np.ones((5,5),np.uint8)
skeletons_small_dilated=[cv2.dilate(skeleton.todense().astype(np.uint8),kernel,iterations = 1) for skeleton in skeletons_small]


In [61]:
plot_t_tp1(node_smalls[t],node_smalls[tp1],poss[t],poss[tp1],skeletons_small_dilated[t],skeletons_small_dilated[tp1],shift=(xbegin,ybegin),time=f't={4*tp1}h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
interest_node = 1351
t=5
tp1=t+1
plot_t_tp1([interest_node],[interest_node],poss[t],poss[tp1],final_pictures[t],final_pictures[t],compress=5,)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
degree4_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)==4] for nx_graph in nx_graph_clean]

In [65]:
numberdegree4=[len(liste) for liste in degree4_nodes]

In [25]:
len(nx_graph_clean[0].edges())

2296

In [66]:
number_anastomosis_theory=[2+len(nx_graph.edges)-len(nx_graph.nodes)-numberdegree4[i] for i,nx_graph in enumerate(nx_graph_clean)]

In [67]:
number_anastomosis_spot = [number_anastomosis_theory[0]]+[number_anastomosis_theory[0]+np.cumsum(number_an)[i] for i in range(len(anastomosiss))] 

In [68]:
per_frame_anastomosis_theory=[number_anastomosis_theory[i+1]-number_anastomosis_theory[i] for i in range(len(number_anastomosis_theory)-1)]

In [69]:
per_frame_anastomosis_spot=[number_anastomosis_spot[i+1]-number_anastomosis_spot[i] for i in range(len(number_anastomosis_theory)-1)]

In [125]:
number_nodes=[len(nx_graph.nodes) for nx_graph in nx_graph_clean]

In [126]:
number_edges=[len(nx_graph.edges) for nx_graph in nx_graph_clean]

In [70]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')
ax.set_ylabel('total number of anastomosis')
theory,=ax.plot(number_anastomosis_theory[:-1])
theory.set_label('theory')
spotting,=ax.plot(number_anastomosis_spot[:-1])
spotting.set_label('frame spotting')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')
ax.set_ylabel('number anastomosis per frame')
theory,=ax.plot(per_frame_anastomosis_theory[:-1])
theory.set_label('theory')
spotting,=ax.plot(per_frame_anastomosis_spot[:-1])
spotting.set_label('frame spotting')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
plt.close('all')

In [33]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(number_nodes[:-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
plot_t_tp1([1093],[1093],poss[t],poss[tp1],final_pictures[tp1],final_pictures[tp1],compress=5,time='4h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
import cv2
i=10
for node in origins[i]:
    node_interest=node
    pos_problem=poss[i][node_interest]
    xbegin=pos_problem[0]-500 
    ybegin=pos_problem[1]-500
    xend=pos_problem[0]+500 
    yend=pos_problem[1]+500
    kernel = np.ones((5,5),np.uint8)
    skeleton_small1=skeletons[i][xbegin:xend,ybegin:yend]
    skeleton_small1=cv2.dilate(skeleton_small1.todense().astype(np.uint8),kernel,iterations = 1)
    skeleton_small2=skeletons[i+1][xbegin:xend,ybegin:yend]
    skeleton_small2=cv2.dilate(skeleton_small2.todense().astype(np.uint8),kernel,iterations = 1)
    plot_t_tp1(origins[i],anastomosiss[i],poss[i],poss[i+1],skeleton_small1,skeleton_small2,time='4h',relabel_tp1=lambda node : find_origin_tip(node,connections[t]), shift=(xbegin,ybegin))

In [73]:
plt.close('all')

In [ ]:
t=3
tp1=t+1
plot_t_tp1([2180],[2180],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [23]:
plot_t_tp1(degree4_nodes[t],degree4_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [35]:
final_pictures_growth = [np.log(make_growth_picture_per_tip(growth_pattern[i],poss[i],factor=500,max_growth=4000,per_tip=True,min_growth=0)[0]+1) for i in range (len(growth_pattern))]


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_pictures_growth[0])

In [27]:
final_pictures[2].shape

(41, 91)

In [37]:
pos_problem=poss[0][444]
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [31]:
import cv2
kernel = np.ones((5,5),np.uint8)


In [40]:
skeletons_small_dilated=[cv2.dilate(skeleton.todense().astype(np.uint8),kernel,iterations = 1) for skeleton in skeletons_small]

In [38]:
t=0
tp1=t+1

In [42]:
plot_t_tp1(node_smalls[tp1],node_smalls[tp1],poss[tp1],poss[tp1],skeletons_small_dilated[tp1],skeletons_small_dilated[tp1],shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save='',time=None):
    right = 0.90
    top = 0.90
    if len(save)>=1:
        fig=plt.figure(figsize=(14,12))
        size = 10
    else:
        fig = plt.figure()
        size = 5
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    ax.text(right, top, time,
        horizontalalignment='right',
        verticalalignment='bottom',
        transform=ax.transAxes,color='white')
    for node in node_list_t:
        t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
                    size=size,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
                        size=size,
                        bbox=bbox_props2)
    if len(save)>=1:
        plt.savefig(save)
        plt.close(fig)
    else:
        plt.show()